In [1]:
import os
import json

import numpy as np
import tensorflow as tf

from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score

os.environ['CUDA_VISIBLE_DEVICES'] = '2'
# solve the problem of "libdevice not found at ./libdevice.10.bc"
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/home/r10222035/.conda/envs/tf2'

2024-12-18 09:27:03.044565: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-18 09:27:03.245556: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def get_highest_accuracy(y_true, y_pred):
    _, _, thresholds = roc_curve(y_true, y_pred)
    # compute highest accuracy
    thresholds = np.array(thresholds)
    if len(thresholds) > 1000:
        thresholds = np.percentile(thresholds, np.linspace(0, 100, 1001))
    accuracy_scores = []
    for threshold in thresholds:
        accuracy_scores.append(accuracy_score(y_true, y_pred>threshold))

    accuracies = np.array(accuracy_scores)
    return accuracies.max()

In [3]:
def get_sample_size(y):
    if len(y.shape) == 1:
        ns = (y == 1).sum()
        nb = (y == 0).sum()
    else:
        ns = (y.argmax(axis=1) == 1).sum()
        nb = (y.argmax(axis=1) == 0).sum()
    print(ns, nb)
    return ns, nb

In [4]:
def get_testing_data(pairing_method):
    config_file = f'config_files/{pairing_method}_config_01.json'

    sig_path = f'./DNN_features/resonant-{pairing_method}.npy'
    bkg_path = f'./DNN_features/pp6b-{pairing_method}.npy'

    with open(config_file, 'r') as f:
        config = json.load(f)
        batch_size = config['batch_size']

    ns, nb = 50000, 50000
    X_s = np.load(sig_path)[:ns]
    X_b = np.load(bkg_path)[:nb]

    r_train, r_val, r_test = 0.7, 0.15, 0.15

    X_s_test = X_s[int(len(X_s)*(r_train+r_val)):]
    X_b_test = X_b[int(len(X_b)*(r_train+r_val)):]

    X_test = np.concatenate((X_s_test, X_b_test), axis=0)
    y_test = np.concatenate((np.ones(len(X_s_test)), np.zeros(len(X_b_test))), axis=0)

    return X_test, y_test

In [5]:
def print_ACC_AUC(model_name, X, y):
    AUC, ACC = [], []
    for i in range(1, 11):

        save_model_name = f'{model_name}_{i:02}/'
        loaded_model = tf.keras.models.load_model(save_model_name)
        y_pred = loaded_model.predict(X, batch_size=2048)
        auc = roc_auc_score(y, y_pred)
        acc = get_highest_accuracy(y, y_pred)
        AUC.append(auc)
        ACC.append(acc)

    AUC = np.array(AUC)
    ACC = np.array(ACC)

    print(f'ACC: {ACC.mean():.4f} ± {ACC.std():.4f}')
    print(f'AUC: {AUC.mean():.4f} ± {AUC.std():.4f}')
    print(f'${ACC.mean():.4f} \pm {ACC.std():.4f}$ & ${AUC.mean():.4f} \pm {AUC.std():.4f}$')

# Training results

## 50k datasets

In [7]:
pairing_method = 'abs_pairing'
X_test, y_test = get_testing_data(pairing_method)

In [8]:
AUC, ACC = [], []
for i in range(1, 11):

    save_model_name = f'DNN_models/last_model_resonant_{pairing_method}_{i:02}/'
    loaded_model = tf.keras.models.load_model(save_model_name)
    X, y = X_test, y_test
    y_pred = loaded_model.predict(X)
    auc = roc_auc_score(y, y_pred)
    acc = get_highest_accuracy(y, y_pred)
    AUC.append(auc)
    ACC.append(acc)

AUC = np.array(AUC)
ACC = np.array(ACC)

print(f'ACC: {ACC.mean():.3f} ± {ACC.std():.3f}')
print(f'AUC: {AUC.mean():.3f} ± {AUC.std():.3f}')

469/469 [==============================] - 1s 2ms/step
ACC: 0.827 ± 0.038
AUC: 0.886 ± 0.048


In [9]:
pairing_method = 'chi2_pairing'
X_test, y_test = get_testing_data(pairing_method)

In [10]:
AUC, ACC = [], []
for i in range(1, 11):

    save_model_name = f'DNN_models/last_model_resonant_{pairing_method}_{i:02}/'
    loaded_model = tf.keras.models.load_model(save_model_name)
    X, y = X_test, y_test
    y_pred = loaded_model.predict(X)
    auc = roc_auc_score(y, y_pred)
    acc = get_highest_accuracy(y, y_pred)
    AUC.append(auc)
    ACC.append(acc)

AUC = np.array(AUC)
ACC = np.array(ACC)

print(f'ACC: {ACC.mean():.3f} ± {ACC.std():.3f}')
print(f'AUC: {AUC.mean():.3f} ± {AUC.std():.3f}')

469/469 [==============================] - 1s 2ms/step
ACC: 0.837 ± 0.003
AUC: 0.900 ± 0.002


In [11]:
pairing_method = '4b_SPANet_pairing'
X_test, y_test = get_testing_data(pairing_method)

In [12]:
AUC, ACC = [], []
for i in range(1, 11):

    save_model_name = f'DNN_models/last_model_resonant_{pairing_method}_{i:02}/'
    loaded_model = tf.keras.models.load_model(save_model_name)
    X, y = X_test, y_test
    y_pred = loaded_model.predict(X)
    auc = roc_auc_score(y, y_pred)
    acc = get_highest_accuracy(y, y_pred)
    AUC.append(auc)
    ACC.append(acc)

AUC = np.array(AUC)
ACC = np.array(ACC)

print(f'ACC: {ACC.mean():.3f} ± {ACC.std():.3f}')
print(f'AUC: {AUC.mean():.3f} ± {AUC.std():.3f}')

469/469 [==============================] - 1s 1ms/step
ACC: 0.829 ± 0.004
AUC: 0.899 ± 0.006


## 100k datasets

In [13]:
pairing_method = 'abs_pairing'
X_test, y_test = get_testing_data(pairing_method)

In [14]:
AUC, ACC = [], []
for i in range(1, 11):

    save_model_name = f'DNN_models/last_model_resonant_4b-{pairing_method}_{i:02}/'
    loaded_model = tf.keras.models.load_model(save_model_name)
    X, y = X_test, y_test
    y_pred = loaded_model.predict(X)
    auc = roc_auc_score(y, y_pred)
    acc = get_highest_accuracy(y, y_pred)
    AUC.append(auc)
    ACC.append(acc)

AUC = np.array(AUC)
ACC = np.array(ACC)

print(f'ACC: {ACC.mean():.3f} ± {ACC.std():.3f}')
print(f'AUC: {AUC.mean():.3f} ± {AUC.std():.3f}')

469/469 [==============================] - 1s 1ms/step
ACC: 0.841 ± 0.003
AUC: 0.903 ± 0.003


In [15]:
pairing_method = 'chi2_pairing'
X_test, y_test = get_testing_data(pairing_method)

In [16]:
AUC, ACC = [], []
for i in range(1, 11):

    save_model_name = f'DNN_models/last_model_resonant_4b-{pairing_method}_{i:02}/'
    loaded_model = tf.keras.models.load_model(save_model_name)
    X, y = X_test, y_test
    y_pred = loaded_model.predict(X)
    auc = roc_auc_score(y, y_pred)
    acc = get_highest_accuracy(y, y_pred)
    AUC.append(auc)
    ACC.append(acc)

AUC = np.array(AUC)
ACC = np.array(ACC)

print(f'ACC: {ACC.mean():.3f} ± {ACC.std():.3f}')
print(f'AUC: {AUC.mean():.3f} ± {AUC.std():.3f}')

469/469 [==============================] - 1s 2ms/step
ACC: 0.838 ± 0.003
AUC: 0.901 ± 0.002


In [17]:
pairing_method = '4b_SPANet_pairing'
X_test, y_test = get_testing_data(pairing_method)

In [ ]:
AUC, ACC = [], []
for i in range(1, 11):

    save_model_name = f'DNN_models/last_model_resonant_4b-{pairing_method}_{i:02}/'
    loaded_model = tf.keras.models.load_model(save_model_name)
    X, y = X_test, y_test
    y_pred = loaded_model.predict(X)
    auc = roc_auc_score(y, y_pred)
    acc = get_highest_accuracy(y, y_pred)
    AUC.append(auc)
    ACC.append(acc)

AUC = np.array(AUC)
ACC = np.array(ACC)

print(f'ACC: {ACC.mean():.3f} ± {ACC.std():.3f}')
print(f'AUC: {AUC.mean():.3f} ± {AUC.std():.3f}')

469/469 [==============================] - 1s 2ms/step
ACC: 0.834 ± 0.006
AUC: 0.905 ± 0.005


: 

## 4b 500k datasets

In [6]:
pairing_method = 'abs_pairing'
model_name = f'DNN_models/last_model_resonant_4b-{pairing_method}'
X_test, y_test = get_testing_data(pairing_method)

print_ACC_AUC(model_name, X_test, y_test)

2024-11-19 15:27:34.481087: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 15:27:35.122989: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:d8:00.0, compute capability: 8.6


 66/469 [===>..........................] - ETA: 0s

2024-11-19 15:27:37.105512: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


469/469 [==============================] - 1s 1ms/step
ACC: 0.844 ± 0.007
AUC: 0.906 ± 0.008


In [7]:
pairing_method = 'chi2_pairing'
model_name = f'DNN_models/last_model_resonant_4b-{pairing_method}'
X_test, y_test = get_testing_data(pairing_method)

print_ACC_AUC(model_name, X_test, y_test)

469/469 [==============================] - 1s 2ms/step
ACC: 0.844 ± 0.003
AUC: 0.908 ± 0.004


In [8]:
pairing_method = '4b_SPANet_pairing'
model_name = f'DNN_models/last_model_resonant_4b-{pairing_method}'
X_test, y_test = get_testing_data(pairing_method)

print_ACC_AUC(model_name, X_test, y_test)

469/469 [==============================] - 1s 2ms/step
ACC: 0.836 ± 0.004
AUC: 0.907 ± 0.004


## 4b 500k datasets, 256 nodes

In [6]:
pairing_method = 'abs_pairing'
model_name = f'DNN_models/last_model_resonant_4b_node_256-{pairing_method}'
X_test, y_test = get_testing_data(pairing_method)

print_ACC_AUC(model_name, X_test, y_test)

2024-11-19 16:05:19.594318: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 16:05:20.198835: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:d8:00.0, compute capability: 8.6


 56/469 [==>...........................] - ETA: 1s

2024-11-19 16:05:22.098015: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


469/469 [==============================] - 1s 3ms/step
ACC: 0.8552 ± 0.0015
AUC: 0.9221 ± 0.0006


In [7]:
pairing_method = 'chi2_pairing'
model_name = f'DNN_models/last_model_resonant_4b_node_256-{pairing_method}'
X_test, y_test = get_testing_data(pairing_method)

print_ACC_AUC(model_name, X_test, y_test)

469/469 [==============================] - 1s 3ms/step
ACC: 0.8537 ± 0.0009
AUC: 0.9212 ± 0.0006


In [8]:
pairing_method = '4b_SPANet_pairing'
model_name = f'DNN_models/last_model_resonant_4b_node_256-{pairing_method}'
X_test, y_test = get_testing_data(pairing_method)

print_ACC_AUC(model_name, X_test, y_test)

469/469 [==============================] - 1s 3ms/step
ACC: 0.8543 ± 0.0009
AUC: 0.9241 ± 0.0003


## 4b 1M datasets, 256 nodes

In [6]:
pairing_method = 'abs_pairing'
model_name = f'DNN_models/last_model_resonant_4b_1M-{pairing_method}'
X_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-{pairing_method}-data.npy')
y_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-{pairing_method}-label.npy')

print_ACC_AUC(model_name, X_test, y_test)

2024-12-03 16:08:41.487551: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 16:08:42.101323: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21847 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:d8:00.0, compute capability: 8.6


49/49 [==============================] - 1s 3ms/step


2024-12-03 16:08:44.000042: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


49/49 [==============================] - 0s 2ms/step
ACC: 0.8651 ± 0.0009
AUC: 0.9317 ± 0.0005


In [7]:
pairing_method = 'chi2_pairing'
model_name = f'DNN_models/last_model_resonant_4b_1M-{pairing_method}'
X_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-{pairing_method}-data.npy')
y_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-{pairing_method}-label.npy')

print_ACC_AUC(model_name, X_test, y_test)

49/49 [==============================] - 0s 2ms/step
ACC: 0.8645 ± 0.0006
AUC: 0.9311 ± 0.0004


In [8]:
pairing_method = '4b_SPANet_pairing'
model_name = f'DNN_models/last_model_resonant_4b_1M-{pairing_method}'
X_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-{pairing_method}-data.npy')
y_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-{pairing_method}-label.npy')

print_ACC_AUC(model_name, X_test, y_test)

49/49 [==============================] - 0s 3ms/step
ACC: 0.8648 ± 0.0004
AUC: 0.9330 ± 0.0004


## 4b mixed mass 1M datasets, 256 nodes

In [6]:
pairing_method = 'abs_pairing'
model_name = f'DNN_models/last_model_resonant_4b_mix_1M-{pairing_method}'
X_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-mix-{pairing_method}-data.npy')
y_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-mix-{pairing_method}-label.npy')

print_ACC_AUC(model_name, X_test, y_test)

2024-12-18 09:27:15.379710: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-18 09:27:16.566897: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:d8:00.0, compute capability: 8.6


49/49 [==============================] - 2s 2ms/step


2024-12-18 09:27:20.783602: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


49/49 [==============================] - 0s 2ms/step
ACC: 0.8262 ± 0.0004
AUC: 0.8968 ± 0.0003
$0.8262 \pm 0.0004$ & $0.8968 \pm 0.0003$


In [7]:
pairing_method = 'chi2_pairing'
model_name = f'DNN_models/last_model_resonant_4b_mix_1M-{pairing_method}'
X_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-mix-{pairing_method}-data.npy')
y_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-mix-{pairing_method}-label.npy')

print_ACC_AUC(model_name, X_test, y_test)

49/49 [==============================] - 0s 2ms/step
ACC: 0.8248 ± 0.0004
AUC: 0.8953 ± 0.0003
$0.8248 \pm 0.0004$ & $0.8953 \pm 0.0003$


In [8]:
pairing_method = '4b_SPANet_pairing'
model_name = f'DNN_models/last_model_resonant_4b_mix_1M-{pairing_method}'
X_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-mix-{pairing_method}-data.npy')
y_test = np.load(f'./DNN_features/triHiggs-4pT40_6b-mix-{pairing_method}-label.npy')

print_ACC_AUC(model_name, X_test, y_test)

49/49 [==============================] - 0s 2ms/step
ACC: 0.8215 ± 0.0008
AUC: 0.8939 ± 0.0003
$0.8215 \pm 0.0008$ & $0.8939 \pm 0.0003$
